In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD, Adadelta, Adam, RMSprop, Adagrad, Nadam, Adamax

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
validation_split = 0.1
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_split, random_state=SEED)

X_train = X_train.astype('float32')
X_train /=255.
X_val = X_val.astype('float32')
X_val /=255.
X_test = X_test.astype('float32')
X_test /=255.

n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_val = to_categorical(y_val, n_classes)
y_test = to_categorical(y_test, n_classes)

In [ ]:
def create_model(opt): 
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

In [ ]:
def create_callbacks(opt):
    callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=2),
             ModelCheckpoint('checkpoints/weights.{epoch:02d}-'+opt+'.h5', save_best_only=False, verbose=True),
             TensorBoard()]
    return callbacks

In [ ]:
opts = dict({
    'sgd': SGD(),
    'sgd-0001': SGD(lr=0.0001, decay=0.00001),
    'adam': Adam(),
    'adam': Adam(lr=0.0001),
    'adadelta': Adadelta(),
    'rmsprop': RMSprop(),
    'rmsprop-0001': RMSprop(lr=0.0001),
    'nadam': Nadam(),
    'adamax': Adamax()
    })

In [ ]:
n_epochs = 1000
batch_size = 128

results = []
# Loop through the optimizers
for opt in opts:
    model = create_model(opt)
    callbacks = create_callbacks(opt)
    model.compile(loss='categorical_crossentropy', optimizer=opts[opt], metrics=['accuracy'])
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=callbacks)
    best_epoch = np.argmax(hist.history['val_acc'])
    best_acc = hist.history['val_acc'][best_epoch] 
    best_model = create_model(opt)
    # Load the model weights with the highest validation accuracy 
    best_model.load_weights('checkpoints/weights.{:02d}-{}.h5'.format(best_epoch, opt))
    best_model.compile(loss='mse', optimizer=opts[opt], metrics=['accuracy'])
    score = best_model.evaluate(X_test, y_test, verbose=0)
    results.append([opt, best_epoch, best_acc, score[1]])

In [ ]:
res = pd.DataFrame(results)
res.columns = ['optimizer', 'epochs', 'val_accuracy', 'test_last', 'test_accuracy']
res